In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [ ]:
# Загрузка и начальная обработка данных
def input_data():
        
        #Подготовка данных и выделение признаков
        data = np.zeros((0,250,250))
        outcome_0 = np.zeros((81,1))
        outcome_1 = np.ones((154,1))
        
        for j in range(2):
            
            i = 1
            if (j == 0): name_part = 'no'; y = 1; n = 82          
            elif (j == 1): name_part = 'yes'; n = 155
                
            for i in range(1,n):
                
                # Загрузка изображения
                s = 'images/brain_tumor_dataset/' + str(name_part) + '/1 (' + str(i) + ').jpg'
                img = cv.imread(s,0)

                # Приведение к одному разрешению
                x_new = 250
                y_new = 250
                dsize = (x_new, y_new)
                output_img = cv.resize(img, dsize, interpolation = cv.INTER_AREA)
                new_img_array = np.array([output_img])  # приведение к 3d массиву

                # Отображение мрт-изображения
                plt.subplots(figsize = (5,5))
                plt.imshow(output_img, cmap = 'gray')
                plt.title('Resizing Image' + str(i)), plt.xticks([]), plt.yticks([])
                plt.show()

                # Преобразование матрицы в строку
                array_data = np.concatenate((data, new_img_array), axis=0)
                data = array_data
                print(s)
        
        
        outcome_0 = np.zeros((81,1))
        outcome_1 = np.ones((154,1))
        array_outcome = np.concatenate((outcome_0, outcome_1), axis=0)
        return array_data, array_outcome

dataset_x, dataset_y = input_data()

In [ ]:
dataset_x.shape

In [ ]:
X, y = shuffle(dataset_x, dataset_y) # Перемешивает в одном порядке массивы (ПРОВЕРИТЬ!!!!!!) ---> проверено!

In [ ]:
X_ = (X - X.mean()) / X.max() # Нормализация данных

In [ ]:
img_0 = X_
img_0 = torch.from_numpy(img_0)
img_0 = torch.reshape(img_0, (235,1,250,250))
img_0 = img_0.float()
img_0.shape

In [ ]:
#conv = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3)(img_0)
#activ = nn.ReLU(inplace=True)(conv)
#pool = nn.MaxPool2d(4)(activ) 
#pred = pool
#pred = pred.view(1,-1)
#pred

In [ ]:
y_data = y
y_data = torch.from_numpy(y_data).float()
y_data = y_data.view(-1,1)
y_data

In [ ]:
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

In [ ]:
class Norma(nn.Module):
    def forward(self, norm):
        norm = (norm - norm.mean()) / norm.max()
        return norm

In [ ]:
CNN_model = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3),
            Norma(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),
            nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),
            Flattener(),  
            nn.Linear(225, 1),
          )

criterion = nn.BCEWithLogitsLoss()  # для бинарной классификации
optimizer = torch.optim.Adam(CNN_model.parameters(), lr=0.01, weight_decay = 0.001)

In [ ]:
epochs = 20
history = []

In [ ]:
for i in range(epochs):
    for j in range(0,235,47):
    
        # 2. compute scores with .forward or .__call__
        logits = CNN_model(img_0[j:j+47,:1,:250,:250])

        # 3. compute loss 
        loss = criterion(logits, y_data[j:j+47,:])
        history.append(loss.item())

        # 4. calc gradients
        optimizer.zero_grad()
        loss.backward()

        # 5. step of gradient descent
        optimizer.step()

    print("Epoches_Step: ", i + 1, "\t", "Loss_Value:", loss.item())

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history)
plt.title('Loss by epoches')
plt.ylabel('Loss')
plt.xlabel('Epoches')
plt.grid()
plt.show()

In [ ]:
# Доля на тренировочных
accuracy = 0.0
for i in range(img_0.shape[0]):
    logits = CNN_model(img_0[i:i+1,:1,:250,:250])

    y_train_predict = logits
    y_train_label = y_data[i:i+1,:].long()

    if (y_train_predict < 0.5):
        y_train_predict = 0
    elif (y_train_predict >= 0.5):
        y_train_predict = 1
    if (y_train_predict == y_train_label):   
        accuracy += 1
    print("Ответ сети: ", y_train_predict, "\t", "Цель: ", y_train_label.item())
        
total_accuracy = accuracy / img_0.shape[0]

In [ ]:
print("Доля верно классифицированных данных: ", total_accuracy)